In [1]:
#library imports
# !pip install hazm
# !pip install fasttext

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
from torch.utils.data import Dataset, DataLoader, Sampler
import torch.nn.functional as F
import torch.optim as optim
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error
# from hazm import *
# import fasttext.util
from torch.nn.utils.rnn import pad_sequence

In [2]:
f = open ('vocab2index.json', "r")
vocab2index = json.loads(f.read())
f = open ('word_vector_dic.json', "r")
word_vector_dic = json.loads(f.read())

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14933 sha256=e9573b9abf61e31f0bbd82400eb296e8134fb5f9e3727727843853de54d48f08
  Stored in directory: /root/.cache/pip/wheels/3d/ec/b0/a96d1d126183f98570a785e6bf8789fca559853a9260e928e1
Successfully built gdown
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  unrar
0 upgraded, 1 newly installed, 0 to remove and 56 not upgraded.
Need to get 113 kB of archives.
After this operation, 406 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 unrar amd64 1:5.6.6-2build1 [113 kB]
Fetched 113 kB in 0s (441 kB/s) 
Selecting previously unselected package unrar.
(Reading database ... 108827 files and directories currently installed.)
Preparing to unpack

In [3]:
train_path = 'data/train.conll'
train_lines = [line for line in open(train_path)]

test_path = 'data/test.conll'
test_lines = [line for line in open(test_path)]

val_path = 'data/dev.conll'
val_lines = [line for line in open(val_path)]

In [4]:
max_lenth = 30
sos = "sos"
eos = "eos"
punc = "PUNC"
def data_reader(lines):
    map_sents = []
    sents, sent = [], []
    all_ezafe_tags = []
    all_pos_tags, pos_tags = [], []
    pos_to_id = {}
    for i, line in enumerate(lines):
            if line != '\n':
                index, word1, word2,pos_tag1, _, _,kasre_rel, pos_tag2, _, _ = line.strip().split('\t')

                sent = sent + [word1]
                if len(map_sents) == 15:
                  print(index, word1, word2,pos_tag1,kasre_rel, pos_tag2)
                
                ezafe_tag = 0
                if lines[i+1] != '\n':
                  ezafe_tag = 1 if index == lines[i+1].strip().split('\t')[6] else 0
                
                if ezafe_tag == 1:
                  pos_tag1 = pos_tag1 + "E"

                if pos_tag1 not in pos_to_id:
                    pos_to_id[pos_tag1] = len(pos_to_id)

                pos_tags = pos_tags + [pos_tag1]
            else:
                sent = [sos] + sent[:max_lenth-3] + ["."] + [eos]
                pos_tags = [sos] + pos_tags[:max_lenth-3] + [punc] + [eos]
                map_sents.append({"sent" : sent,"pos_tags" : pos_tags})
                ezafe_tags = []
                pos_tags = []
                sent = []

    map_sents = list(sorted(map_sents, key=lambda d: len(d['sent']), reverse=True)) 
    return map_sents

train_map_sents = data_reader(train_lines)
test_map_sents = data_reader(test_lines)
val_map_sents = data_reader(val_lines)

1 سالهاست سالهاست V 1 V
2 که که CONJ 2 CONJ
3 کشورهای کشورهای N 3 N
4 پیشرفته پیشرفته AJ 3 AJ
5 و و CONJ 5 CONJ
6 در در P 6 P
7 حال حال N 7 N
8 توسعه توسعه N 7 N
9 جهان جهان N 8 N
10 استفاده استفاده N 10 N
11 از از P 11 P
12 روش روش N 12 N
13 یک یک NUM 12 NUM
14 کتابی کتابی AJ 14 AJ
15 را را POSTP 15 POSTP
16 در در P 16 P
17 تدریس تدریس N 17 N
18 ترک ترک N 18 N
19 گفته گفته V 19 V
20 و و CONJ 20 CONJ
21 راه راه N 21 N
22 مواد مواد N 21 N
23 خواندنی خواندنی AJ 22 AJ
24 بی‌شمار بی‌شمار AJ 23 AJ
25 دیگری دیگری PRO 24 PRO
26 را را POSTP 26 POSTP
27 برای برای P 27 P
28 تکمیل تکمیل N 27 N
29 برنامه برنامه N 28 N
30 درسی درسی AJ 29 AJ
31 به به P 31 P
32 کلاس کلاس N 32 N
33 باز باز AJ 33 AJ
34 کرده‌اند کرده‌اند V 34 V
35 . . PUNC 35 PUNC
1 این این DET 1 DET
2 مدرسه مدرسه N 2 N
3 در در P 3 P
4 منطقه منطقه N 4 N
5 14 14 NUM 4 NUM
6 آموزش آموزش N 5 N
7 و و CONJ 7 CONJ
8 پرورش پرورش N 8 N
9 قرار قرار N 9 N
10 دارد دارد V 10 V
11 . . PUNC 11 PUNC
1 نقد نقد N 1 N
2 فیلم فیلم N 1 N
3 » » PUNC 3 PUNC


In [5]:
df_train = pd.DataFrame.from_dict(train_map_sents)
df_test = pd.DataFrame.from_dict(test_map_sents)
df_val = pd.DataFrame.from_dict(val_map_sents)

all_words = []
def fetch_words(sent):
  all_words.extend(sent)

df_train.apply(lambda row : fetch_words(row['sent']), axis = 1)
df_test.apply(lambda row : fetch_words(row['sent']), axis = 1)
df_val.apply(lambda row : fetch_words(row['sent']), axis = 1)
all_words = list(set(all_words))
print(len(all_words))


100820


In [6]:
index = 15
for i in range(len(df_train["sent"][index])):
  print(df_train["sent"][index][i],df_train["pos_tags"][index][i])

sos sos
روشهای NE
مختلف AJ
در P
علاقمند AJ
کردن NE
کودکان N
به P
مطالعه NE
نظام NE
تعلیم N
و CONJ
تربیت N
، PUNC
روشهای NE
تدریس NE
کتابهای NE
درسی AJ
و CONJ
نحوه NE
کار NE
معلم N
با P
وسایل NE
ارتباط N
جمعی AJ
در P
خانواده N
. PUNC
eos eos


In [7]:
all_tags = []
def fetch_pos(pos_tags):
  all_tags.extend(pos_tags)

df_train.apply(lambda row : fetch_pos(row['pos_tags']), axis = 1)
df_test.apply(lambda row : fetch_pos(row['pos_tags']), axis = 1)
df_val.apply(lambda row : fetch_pos(row['pos_tags']), axis = 1)

all_tags = all_tags + ["PAD"]
all_tags = list(set(all_tags))
pos2index = {k: v for v, k in enumerate(all_tags)}
print(len(all_tags))
print(pos2index)

27
{'DET': 0, 'PUNC': 1, 'sos': 2, 'DETE': 3, 'PAD': 4, 'NE': 5, 'V': 6, 'RESE': 7, 'AJE': 8, 'INT': 9, 'ADVE': 10, 'AJCC': 11, 'N': 12, 'RES': 13, 'PRO': 14, 'AJ': 15, 'POSTP': 16, 'CL': 17, 'P': 18, 'CONJE': 19, 'CONJ': 20, 'NUM': 21, 'PE': 22, 'NUME': 23, 'eos': 24, 'ADV': 25, 'PROE': 26}


In [8]:
def encode_sentence(text, vocab2index, N):
    encoded = np.full((N), vocab2index["PAD"])
    enc1 = np.array([vocab2index[word] for word in text])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length


df_train['encoded_sent'] = df_train['sent'].apply(lambda x: np.array(encode_sentence(x,vocab2index,max_lenth)))
df_test['encoded_sent'] = df_test['sent'].apply(lambda x: np.array(encode_sentence(x,vocab2index,max_lenth)))
df_val['encoded_sent'] = df_val['sent'].apply(lambda x: np.array(encode_sentence(x,vocab2index,max_lenth)))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == "__main__":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  # Remove the CWD from sys.path while we load stuff.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you me

In [9]:
def encode_pos(pos_tags, pos2index, N):
    encoded = np.full((N), pos2index["PAD"], dtype=int)
    enc1 = np.array([pos2index[pos_tag] for pos_tag in pos_tags])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

df_train['encoded_pos'] = df_train['pos_tags'].apply(lambda x: np.array(encode_pos(x,pos2index,max_lenth)))
df_test['encoded_pos'] = df_test['pos_tags'].apply(lambda x: np.array(encode_pos(x,pos2index,max_lenth)))
df_val['encoded_pos'] = df_val['pos_tags'].apply(lambda x: np.array(encode_pos(x,pos2index,max_lenth)))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == "__main__":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when

In [10]:
index = 1
print(pos2index["PAD"])
print(df_train['sent'][index])
print(df_train['encoded_pos'][index])

4
['sos', 'مجموعه', 'اظهارات', 'مسئولان', 'و', 'بررسیهای', 'گوناگون', 'که', 'از', 'سوی', 'بعضی', 'پژوهشگران', 'و', 'علاقه', 'مندان', 'در', 'زمینه', 'کتابخانه\u200cهای', 'آموزشگاهی', 'به', 'عمل', 'آمده', 'است', '،', 'نشانگر', 'کمبودها', 'و', 'نارسائیهای', '.', 'eos']
[array([ 2,  5,  5, 12, 20,  5, 15, 20, 18,  5,  0, 12, 20, 12, 12, 18,  5,
         5, 15, 18, 12,  6,  6,  1,  5, 12, 20,  5,  1, 24])
 30]


In [11]:
SEED = 1234
import random
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [13]:
import math
import pickle
import random

class SentDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), torch.from_numpy(self.y[idx][0].astype(np.int32)), torch.tensor(max_lenth, dtype=torch.int32)

In [14]:
from sklearn.model_selection import train_test_split

BATCH_SIZE = 1024

train_dataset = SentDataset(df_train['encoded_sent'], df_train['encoded_pos'])
valid_dataset = SentDataset(df_val['encoded_sent'], df_val['encoded_pos'])
test_dataset = SentDataset(df_test['encoded_sent'], df_test['encoded_pos'])

print(len(train_dataset))
print(len(valid_dataset))
print(len(test_dataset))


train_dl = DataLoader(train_dataset, batch_size=BATCH_SIZE,num_workers=2,pin_memory=True, shuffle=False)
val_dl = DataLoader(valid_dataset, batch_size=BATCH_SIZE,num_workers=2,pin_memory=True,)
test_dl = DataLoader(test_dataset, batch_size=BATCH_SIZE,num_workers=2,pin_memory=True,)


del(train_lines,test_lines,val_lines)
del(train_map_sents,test_map_sents,val_map_sents)
del(train_dataset,valid_dataset,test_dataset)
del(all_words,all_tags)


304798
19046
21715


In [15]:
print(len(word_vector_dic.keys()))
print(len(vocab2index.keys()))
print(vocab2index["PAD"])

117164
117166
0


In [16]:
def get_emb_matrix(emb_size = 300):
    """ Creates embedding matrix from word vectors"""
    vocab_size = len(word_vector_dic.keys()) + 2
    vocab_to_idx = {}
    vocab = ["PAD", "UNK"]
    W = np.zeros((vocab_size, emb_size), dtype="float32")
    W[0] = np.zeros(emb_size, dtype='float32') # adding a vector for padding
    W[1] = np.random.uniform(-0.25, 0.25, emb_size) # adding a vector for unknown words 
    vocab_to_idx["PAD"] = 0
    vocab_to_idx["UNK"] = 1
    i = 2
    for word in word_vector_dic.keys():
        if word in word_vector_dic:
            W[i] = word_vector_dic[word]
        else:
            print(word)
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
        vocab_to_idx[word] = i
        vocab.append(word)
        i += 1   
    return W, vocab_to_idx, vocab_size

pretrained_weights, vocab2index, vocab_size = get_emb_matrix(emb_size = 300)

In [17]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, pad_idx, weights):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx = pad_idx)
        self.embedding.weight.data.copy_(torch.from_numpy(weights).cuda())
        self.embedding.weight.requires_grad = False
        
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_len):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
                
        #need to explicitly put lengths on cpu!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.to('cpu'))
                
        packed_outputs, hidden = self.rnn(packed_embedded)
                                 
        #packed_outputs is a packed sequence containing all hidden states
        #hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
        #outputs is now a non-packed sequence, all hidden states obtained
        #  when the input is a pad token are all zeros
            
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
  
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention = [batch size, src len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return F.softmax(attention, dim = 1)
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs, mask):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        #mask = [batch size, src len]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs, mask)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0), a.squeeze(1)




class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.device = device
        
    def create_mask(self, src):
        mask = (src != self.src_pad_idx).permute(1, 0)
        return mask
        
    def forward(self, src, src_len, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #src_len = [batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
                    
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        tag_preds = torch.zeros(trg_len, batch_size).to(self.device)
        tag_labels = torch.zeros(trg_len, batch_size).to(self.device)
        

        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src, src_len)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        mask = self.create_mask(src)

        #mask = [batch size, src len]
                
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            #  and mask
            #receive output tensor (predictions) and new hidden state
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs, mask)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            tag_preds[t] = top1
            
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
            tag_labels[t] = trg[t]
            
        return outputs, torch.flatten(tag_labels), torch.flatten(tag_preds)


In [18]:
# from tqdm import tqdm
from time import sleep
import gc

def train(model, iterator, optimizer, criterion, clip, print_every):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    # tepoch = tqdm(iterator)
    for i,(x, y, l) in enumerate(iterator):
        text = x.long().T.cuda()
        tags = y.long().T.cuda()
        src_len = l
        
        optimizer.zero_grad()
        # for param in model.parameters():
          # param.grad = None
        
        #text = [sent len, batch size]
        predictions, tag_labels, tag_preds = model(text, src_len, tags, 1)
        
        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.reshape(-1)
        
        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        loss = criterion(predictions, tags)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        # tepoch.set_postfix(loss=loss.item())
        # sleep(0.1)

    return epoch_loss / len(iterator)

# from sklearn.metrics import precision_recall_fscore_support as score
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    # y_preds = []
    # y_trg = []
    with torch.no_grad():

        for i, (x, y, l) in enumerate(iterator):
            src = x.long().T.to(device)
            trg = y.long().T.to(device)
            src_len = l.cuda()

            output, tag_labels, tag_preds = model(src, src_len, trg, 0)
        
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output.view(-1, output_dim)
            trg = trg.reshape(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]
          
            loss = criterion(output, trg.cuda())
            
            epoch_loss += loss.item()
            
            # y_preds = y_preds + list(tag_labels.cpu())
            # y_trg = y_trg + list(tag_preds.cpu())
            # precision,recall,fscore,support=score(y_trg,y_preds)
            # print("F1:",fscore)

        
    return epoch_loss / len(iterator) 

def test(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    y_preds = []
    y_trg = []
    with torch.no_grad():

        for i, (x, y, l) in enumerate(iterator):
            if i%10 == 0:
              print(i)
              
            src = x.long().T.to(device)
            trg = y.long().T.to(device)
            src_len = l.cuda()

            output, tag_labels, tag_preds = model(src, src_len, trg, 1)

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output.view(-1, output_dim)
            trg = trg.reshape(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            y_preds = y_preds + list(tag_labels.cpu())
            y_trg = y_trg + list(tag_preds.cpu())

            loss = criterion(output, trg.cuda())
            
            epoch_loss += loss.detach().item()
        
    return epoch_loss / len(iterator) , y_preds, y_trg

In [19]:
import time
import math
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since):
    now = time.time()
    s = now - since
    return asMinutes(s)

In [20]:
INPUT_DIM = len(vocab2index.keys())
OUTPUT_DIM = len(pos2index.keys())
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
ENC_HID_DIM = 800
DEC_HID_DIM = 800
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
SRC_PAD_IDX = vocab2index["PAD"]
TRG_PAD_IDX = pos2index["PAD"]

print(INPUT_DIM)
print(OUTPUT_DIM)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM).to(device)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT, SRC_PAD_IDX, pretrained_weights).to(device)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn).to(device)

model = Seq2Seq(enc, dec, SRC_PAD_IDX, device).to(device)

117166
27


In [21]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 15,057,827 trainable parameters


In [22]:
lr = 0.01
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [23]:
import time
import gc
from datetime import datetime

N_EPOCHS = 10
CLIP = 0.1

# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
print(len(train_dl))
best_valid_loss = float('inf')
start_time = time.time()
for epoch in range(N_EPOCHS): 
    gc.collect()
    train_loss = train(model, train_dl, optimizer, criterion, CLIP,30)
    valid_loss = evaluate(model, val_dl, criterion) 
    
    print(timeSince(start_time))
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Val. Loss: {valid_loss:.3f}')
    
test_loss, y_preds, y_trg = test(model, test_dl, criterion)
print(classification_report(y_preds,y_trg))

298
9m 37s
Epoch: 01, Train Loss: 1.779, Val. Loss: 8.635
19m 13s
Epoch: 02, Train Loss: 1.619, Val. Loss: 8.442
28m 47s
Epoch: 03, Train Loss: 1.421, Val. Loss: 5.729
38m 22s
Epoch: 04, Train Loss: 1.006, Val. Loss: 4.727
47m 57s
Epoch: 05, Train Loss: 0.898, Val. Loss: 3.407
57m 31s
Epoch: 06, Train Loss: 0.758, Val. Loss: 1.381
67m 8s
Epoch: 07, Train Loss: 0.685, Val. Loss: 0.949
76m 43s
Epoch: 08, Train Loss: 0.640, Val. Loss: 0.960
86m 18s
Epoch: 09, Train Loss: 0.605, Val. Loss: 0.860
95m 52s
Epoch: 10, Train Loss: 0.573, Val. Loss: 1.427
0
10
20


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95     32462
         1.0       0.60      0.91      0.72     62929
         3.0       0.85      0.67      0.75      2224
         4.0       0.00      0.00      0.00    116332
         5.0       0.56      0.88      0.68     89424
         6.0       0.61      0.82      0.70     39709
         7.0       0.69      0.40      0.51       171
         8.0       0.70      0.34      0.46     10970
         9.0       0.92      0.60      0.72        57
        10.0       0.87      0.78      0.82       752
        11.0       0.00      0.00      0.00        11
        12.0       0.64      0.56      0.60    103980
        13.0       0.79      0.51      0.62      2696
        14.0       0.87      0.77      0.81     11637
        15.0       0.45      0.61      0.52     30505
        16.0       0.77      0.84      0.80      6088
        17.0       0.80      0.51      0.63      1673
        18.0       0.89    

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
def test(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    y_preds = []
    y_trg = []
    with torch.no_grad():

        for i, (x, y, l) in enumerate(iterator):
            if i%10 == 0:
              print(i)
              
            src = x.long().T.to(device)
            trg = y.long().T.to(device)
            src_len = l.cuda()

            output, tag_labels, tag_preds = model(src, src_len, trg, 1)

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output.view(-1, output_dim)
            trg = trg.reshape(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            y_preds = y_preds + list(tag_labels.cpu())
            y_trg = y_trg + list(tag_preds.cpu())

            loss = criterion(output, trg.cuda())
            
            epoch_loss += loss.detach().item()
        
    return epoch_loss / len(iterator) , y_preds, y_trg

In [25]:
from sklearn.metrics import classification_report
test_loss, y_preds, y_trg = test(model, test_dl, criterion)
print(classification_report(y_preds,y_trg))
# print("----------------------")
# lcm = np.lcm(y_preds,y_trg)
# print("lcm :" + str(lcm))

0
10
20


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95     32462
         1.0       0.60      0.91      0.72     62929
         3.0       0.85      0.67      0.75      2224
         4.0       0.00      0.00      0.00    116332
         5.0       0.56      0.88      0.68     89424
         6.0       0.61      0.82      0.70     39709
         7.0       0.69      0.40      0.51       171
         8.0       0.70      0.34      0.46     10970
         9.0       0.92      0.60      0.72        57
        10.0       0.87      0.78      0.82       752
        11.0       0.00      0.00      0.00        11
        12.0       0.64      0.56      0.60    103980
        13.0       0.79      0.51      0.62      2696
        14.0       0.87      0.77      0.81     11637
        15.0       0.45      0.61      0.52     30505
        16.0       0.77      0.84      0.80      6088
        17.0       0.80      0.51      0.63      1673
        18.0       0.89    

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
